In [3]:
import json
from transformers import pipeline
import re

with open('cnn_articles.json', 'r') as file:
    original_json = json.load(file)

def extract_labels(articles):
    labels = set()
    for article in articles:
        words = re.findall(r'\b\w+\b', article['title'])
        for word in words:
            labels.add(word)
    return list(labels)

candidate_labels = extract_labels(original_json)
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
new_json = {
    "documents": []
}

for article in original_json['articles']:
    text = article['title'] + " " + " ".join(article['body'])
    classification = classifier(text, candidate_labels)
    topic = classification['labels'][0]
    concept = {
        "concept": article['title'],
        "content": " ".join(article['body']) 
    }
    topic_exists = False
    for document in new_json["documents"]:
        if document["topic"] == topic:
            document["concepts"].append(concept)
            topic_exists = True
            break
    if not topic_exists:
        new_json["documents"].append({
            "topic": topic,
            "concepts": [concept]
        })
new_json_str = json.dumps(new_json, indent=4)
with open('converted_cnn_articles.json', 'w') as file:
    file.write(new_json_str)


KeyboardInterrupt: 